In [1]:
data = open('2023_22.txt', 'r').read()

data1 = '''1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9'''

# load data into a 2D array of integers
# replace ~ with comma and split on comma\
data = data.replace("~",",")
data3D = [[int(num) for num in row.split(",")] for row in data.split("\n") if row != ""]

def xyoverlap(x1,y1,x2,y2,x3,y3,x4,y4):
    # check if two rectangles overlap in the x and y dimensions
    if ranges_overlap(x1, x2, x3, x4) & ranges_overlap(y1, y2, y3, y4):
        return True
    else:
        return False
    
def ranges_overlap(p1, p2, p3, p4):
    # check if [p1,p2] and [p3,p4] overlap
    return p1 <= p4 and p3 <= p2

overlaps = {}
for i in range(len(data3D)):
    overlaps[i] = []
    for j in range(len(data3D)):
        if i == j:
            continue
        else:
            # check if rectangles overlap
            if xyoverlap(data3D[i][0], data3D[i][1], data3D[i][3], data3D[i][4], data3D[j][0], data3D[j][1], data3D[j][3], data3D[j][4]):
                overlaps[i].append(j)

grounded = [False for i in range(len(data3D))]
# while any blocks are not grounded

for i in range(len(data3D)):
    if data3D[i][2] == 1:
        grounded[i] = True

while not all(grounded):
    for i in range(len(data3D)):
        if not grounded[i]:
            maxbelow = 0
            belowgrounded = True
            for j in overlaps[i]:
                if data3D[j][5]<data3D[i][2]:
                    if data3D[j][5] > maxbelow:
                        maxbelow = data3D[j][5]
                        belowgrounded = grounded[j]
            height = data3D[i][5] - data3D[i][2]
            data3D[i][2] = maxbelow + 1
            data3D[i][5] = data3D[i][2] + height
            grounded[i] = belowgrounded

supportedby = {}
for i in range(len(data3D)):
    supportedby[i] = []
    if data3D[i][2] == 1:
        continue
    for j in overlaps[i]:
        if data3D[j][5] == data3D[i][2]-1:
            supportedby[i].append(j)
            
supports = {}
for i in range(len(data3D)):
    supports[i] = []
    for j in overlaps[i]:
        if data3D[j][2] == data3D[i][5]+1:
            supports[i].append(j)
       
vaper=0
for i in range(len(data3D)):
    cankill = True
    for j in supports[i]:
        if len(supportedby[j]) == 1:
            cankill = False
            break
    if cankill:
        vaper +=1
            
    
print('Part 1:',vaper)
       



Part 1: 375


In [2]:
import numpy as np

# create matrix of size len(data3D)+1 x len(data3D)+1
# M[i][j] == 1 if block i is supported by block j (or is grounded)
M = np.zeros((len(data3D)+1,len(data3D)+1))
for i in range(len(data3D)):
    for j in supportedby[i]:
        M[i][j] = 1
    # if block is grounded, add 1 to the last column
    if len(supportedby[i]) == 0:
        M[i][len(data3D)] = 1
# the ground is supported by itself
M[len(data3D)][len(data3D)] = 1
fall_count=0
for i in range(len(data3D)):
    notgone = np.ones((len(data3D)+1, 1))
    Mcopy = M.copy()
    Mcopy[i] == 0
    notgone[i] = 0
    notfell = M @ notgone
    # while there are blocks that have fallen that we have not noted
    justfallen = (notfell>0) != (notgone>0)
    while justfallen.any():
        for j in range(len(data3D)):
            if justfallen[j]:
                notgone[j] = 0
                Mcopy[j] = 0
        notfell = Mcopy @ notgone
        justfallen = (notfell>0) != (notgone>0)
    fall_count += len(data3D)-notgone.sum()

print('Part 2:',round(fall_count))

        

Part 2: 72352
